In [1]:
import networkx as nx
import heapq as hq
from copy import deepcopy

# For some reason pylance doesn't like these imports, but they work fine
import KnowledgeGraph as kg  # type: ignore
import ConceptNode as cn  # type: ignore
from MultiDistance import MultiDistance, multimin, sum_pareto_distance  # type: ignore

In [2]:
G0=nx.read_gpickle('Sample DDI networks/net-ddi-bnu-tau.gpickle')
G1=nx.read_gpickle('Sample DDI networks/net-ddi-cat-tau.gpickle')
G2=nx.read_gpickle('Sample DDI networks/net-ddi-indy-tau.gpickle')
Layers = {'BNU':G0,'CAT':G1,'IND':G2}

In [3]:
all_nodes = set([*list(G0.nodes()),*list(G1.nodes()),*list(G2.nodes())])

for node in all_nodes:
    for G in Layers.values():
        if node not in G.nodes():
            G.add_node(node)

set(Layers['BNU'].nodes())==set(Layers['CAT'].nodes())==set(Layers['IND'].nodes())

True

In [34]:
print(f'{len(all_nodes)=}')
print(f'{Layers["BNU"].number_of_edges()=}')
print(f'{Layers["CAT"].number_of_edges()=}')
print(f'{Layers["IND"].number_of_edges()=}')
print(f'total edges={Layers["BNU"].number_of_edges()+Layers["CAT"].number_of_edges()+Layers["IND"].number_of_edges()}')


len(all_nodes)=381
Layers["BNU"].number_of_edges()=146
Layers["CAT"].number_of_edges()=708
Layers["IND"].number_of_edges()=776
total edges=1630


In [5]:
KG = kg.KnowledgeGraph(Layers)

In [23]:
removed_edges = KG.pareto_backbone_removed_edges(depth_cut=5)

In [35]:
print(removed_edges)

{(('DB00502', 'CAT'), ('DB00196', 'CAT')), (('DB00252', 'CAT'), ('DB00338', 'CAT')), (('DB01167', 'CAT'), ('DB00199', 'CAT')), (('DB01045', 'CAT'), ('DB00254', 'CAT')), (('DB00759', 'CAT'), ('DB01060', 'CAT')), (('DB01211', 'CAT'), ('DB00277', 'CAT')), (('DB01045', 'CAT'), ('DB01211', 'CAT')), (('DB00564', 'CAT'), ('DB01418', 'CAT')), (('DB00254', 'CAT'), ('DB01045', 'CAT')), (('DB01076', 'CAT'), ('DB00564', 'CAT')), (('DB01211', 'CAT'), ('DB01394', 'CAT')), (('DB00215', 'CAT'), ('DB01418', 'CAT')), (('DB00404', 'CAT'), ('DB00252', 'CAT')), (('DB00564', 'CAT'), ('DB00641', 'CAT')), (('DB00252', 'CAT'), ('DB01118', 'CAT')), (('DB00766', 'IND'), ('DB00254', 'IND')), (('DB01211', 'CAT'), ('DB01045', 'CAT')), (('DB01045', 'CAT'), ('DB00196', 'CAT')), (('DB00091', 'CAT'), ('DB00196', 'CAT')), (('DB00641', 'CAT'), ('DB00564', 'CAT')), (('DB01167', 'CAT'), ('DB01211', 'CAT')), (('DB00682', 'CAT'), ('DB00564', 'CAT')), (('DB01242', 'CAT'), ('DB01367', 'CAT')), (('DB00564', 'CAT'), ('DB01076', 

In [8]:
import distanceclosure as dc

In [28]:
print(G0.number_of_edges())
print(len(G0.edges()))
print(nx.DiGraph(G0).number_of_edges())

146
146
292


In [60]:
BB0=dc.distance_closure(G0,only_backbone=True)
BB1=dc.distance_closure(G1,only_backbone=True)
BB2=dc.distance_closure(G2,only_backbone=True)

removed_edges_BB = []
for BB,L in zip( [BB0,BB1,BB2], ['BNU','CAT','IND']):
    for u,v,d in list(BB.edges(data=True)):
        if not d['is_trig']:
            removed_edges_BB.append(((u,L),(v,L)))
            BB.remove_edge(u,v)


In [63]:
sre = set(removed_edges | set((v,u) for u,v in removed_edges))
sreBB = set(removed_edges_BB+[(v,u) for u,v in removed_edges_BB])
print(sreBB-sre)
print(sre-sreBB)
print(len(sreBB)/2)
print(len(sre)/2)

{(('DB00857', 'CAT'), ('DB00514', 'CAT')), (('DB00514', 'CAT'), ('DB00857', 'CAT'))}
set()
53.0
52.0


In [12]:
LayersBB = {'BNU':BB0,'CAT':BB1,'IND':BB2}

In [13]:
all_nodesBB = set([*list(BB0.nodes()),*list(BB1.nodes()),*list(BB2.nodes())])

In [14]:
for node in all_nodesBB:
    for G in LayersBB.values():
        if node not in G.nodes():
            G.add_node(node)

set(LayersBB['BNU'].nodes())==set(LayersBB['CAT'].nodes())==set(LayersBB['IND'].nodes())

True

In [15]:
print(f'{len(all_nodesBB)=}')
print(f'{len(LayersBB["BNU"].edges())=}')
print(f'{len(LayersBB["CAT"].edges())=}')
print(f'{len(LayersBB["IND"].edges())=}')
print(f'{len(LayersBB["BNU"].edges())+len(LayersBB["CAT"].edges())+len(LayersBB["IND"].edges())=}')


len(all_nodesBB)=381
len(LayersBB["BNU"].edges())=143
len(LayersBB["CAT"].edges())=660
len(LayersBB["IND"].edges())=774
len(LayersBB["BNU"].edges())+len(LayersBB["CAT"].edges())+len(LayersBB["IND"].edges())=1577


In [16]:
KGBB = kg.KnowledgeGraph(LayersBB)

In [17]:
cBB=KGBB.pareto_distance_closure(depth_cut=3,start_layer="IND")

In [18]:
KGBB.pareto_backbone_removed_edges(closure=cBB)

set()